In [26]:
import os
from pkg import u

clen_values, photon_energy_values = [1.5, 2.5, 3.5], [6000, 7000, 8000]
param_matrix = u.parameter_matrix(clen_values, photon_energy_values)
print(param_matrix)

dataset_dict = {
    '01': [clen_values[0], photon_energy_values[0]],
    '02': [clen_values[0], photon_energy_values[1]],
    '03': [clen_values[0], photon_energy_values[2]],
    '04': [clen_values[1], photon_energy_values[0]],
    '05': [clen_values[1], photon_energy_values[1]],
    '06': [clen_values[1], photon_energy_values[2]],
    '07': [clen_values[2], photon_energy_values[0]],
    '08': [clen_values[2], photon_energy_values[1]],
    '09': [clen_values[2], photon_energy_values[2]],
}


# parameters
dataset = '06'
print(dataset_dict[dataset])
clen, photon_energy = dataset_dict[dataset]
threshold = 1
# clen = 1.5 # meters 
# photon_energy = 6000 # eV/ 6 keV

# instances
pm = u.PathManager()
peak_paths, water_peak_paths, labels, water_background_path = pm.select_dataset(dataset=dataset)
p = u.Processor(paths=pm, dataset=dataset)
dm = u.DatasetManager(paths=pm, dataset=dataset, transform=None)


true parameter matrix...

[[(1.5, 6000.) (1.5, 7000.) (1.5, 8000.)]
 [(2.5, 6000.) (2.5, 7000.) (2.5, 8000.)]
 [(3.5, 6000.) (3.5, 7000.) (3.5, 8000.)]]
[2.5, 8000]
Found water background image: water06.h5
Number of peak images: 19
Number of water images: 0
Number of label images: 0
Check: Path to water background image: /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/water/06/water06.h5



In [27]:
# p.process_directory(dataset=dataset, clen=clen, photon_energy=photon_energy)

'dataset' value 06 confirmed successfully.
'clen' value 2.5 confirmed successfully.
'photon_energy' value 8000 confirmed successfully.
'water_background' value /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/water/06/water06.h5 confirmed successfully.
Attributes 'clen' and 'photon_energy' updated for /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/06/img_8keV_clen02_00114.h5
Attributes 'clen' and 'photon_energy' updated for /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks_water_overlay/06/overlay_img_8keV_clen02_00114.h5
Attributes 'clen' and 'photon_energy' updated for /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/labels/06/label_img_8keV_clen02_00114.h5
Processed and labeled images for img_8keV_clen02_00114.h5 saved.
Attributes 'clen' and 'photon_energy' updated for /Users/adamkurth/Documents/vscode/CXFEL/cxls_hitfinder/images/peaks/06/img_8keV_clen02_00104.h5
Attributes 'clen' and 'photon_energy' updated for /Users

In [2]:
# generate peak/overlay/label and upate attributes
# overlaying the correct water01.h5 path to the peak images
# p.process_directory(dataset=dataset, clen=clen, photon_energy=photon_energy)

# peak, label, overlay, background are valid types
u.check_attributes(paths=pm, dataset=dataset, type='label') 

train_loader, test_loader = u.prepare(data_manager=dm, batch_size=10)


true parameter matrix...

All files in dataset 01 of type 'label' have matching attributes.

Data prepared.
Train size: 31
Test size: 8
Batch size: 10
Number of batches in train_loader: 4 




true parameter matrix...

All files in dataset 01 of type 'label' have matching attributes.


True